In [1]:
from training_datasets.GLUEMetaDataset import GLUEMetaDataset
from samplers.FewShotEpisodeSampler import FewShotEpisodeSampler
from samplers.FewShotEpisodeBatchSampler import FewShotEpisodeBatchSampler
import torch.utils.data as data

meta_ds = GLUEMetaDataset(k=4,numTasks=1000)

/home/aksingh/miniconda3/envs/meta-learned-lines/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset glue (/home/aksingh/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
100%|███████████████████████████████████████████| 3/3 [00:00<00:00, 1289.36it/s]
Found cached dataset glue (/home/aksingh/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
100%|████████████████████████████████████████████| 5/5 [00:00<00:00, 732.32it/s]
Found cached dataset glue (/home/aksingh/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
100%|███████████████████████████████████████████| 3/3 [00:00<00:00, 1436.73it/s]
Found cached dataset 

In [2]:
train_protomaml_sampler = FewShotEpisodeBatchSampler(meta_ds, kShot=4, batchSize=4)

train_protomaml_loader = data.DataLoader(
    meta_ds, batch_sampler=train_protomaml_sampler, collate_fn=train_protomaml_sampler.getCollateFunction(), num_workers=4)

# for i in range(1):
#     batch = next(iter(train_protomaml_loader))
#     for episode_i in range(len(batch[0])):
#         data, labels = batch[0][episode_i], batch[1][episode_i]
#         supportSet, supportLabels = data[0:len(data)//2], labels[0:len(data)//2] 
#         print(supportLabels)

In [3]:
# episodes = []
# for i in range(1):
#     episodes.append(gm_ds.getTask())
# for i in range(len(episodes)):
#     classes = len(set(episodes[i][1].tolist()))
#     # print(classes)
#     print(episodes[i][0], episodes[i][1], "\n")

In [4]:
# from training.models.ProtoFOMAML import ProtoFOMAML

# pfomaml = ProtoFOMAML(outerLR=5e-4, innerLR=1e-3, outputLR=1e-2, steps=5, batchSize=16, warmupSteps=0)
# pfomaml.training_step(next(iter(train_protomaml_loader)), 0)

In [5]:
from samplers.FewShotValidationEpisodeBatchSampler import FewShotValidationEpisodeBatchSampler
from samplers.FewShotValidationEpisodeSampler import FewShotValidationEpisodeSampler
from validation_datasets.ValidationDataset import ValidationDataset
import torch.utils.data as data

ds = ValidationDataset()

val_protomaml_sampler = FewShotValidationEpisodeBatchSampler(ds, kShot=4)
val_protomaml_loader = data.DataLoader(
    ds, batch_sampler=val_protomaml_sampler, collate_fn=val_protomaml_sampler.getCollateFunction(), num_workers=1
)

for i in range(1):
    batch = next(iter(val_protomaml_loader))
    for episode_i in range(len(batch[0])):
        data, labels = batch[0][episode_i], batch[1][episode_i]
        supportSet, supportLabels = data[0:len(data)//2], labels[0:len(data)//2] 
#         print(supportLabels)
#         print(labels)

In [6]:
from training.models.ProtoFOMAML import ProtoFOMAML
from training.models.ProtoNet import ProtoNet

# pfomaml = ProtoFOMAML(outerLR=5e-4, innerLR=1e-3, outputLR=1e-2, steps=5, batchSize=16, warmupSteps=0)
# pfomaml.validation_step(next(iter(val_protomaml_loader)), 0)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
from training.trainer import train_model
import torch.utils.data as data

protomaml_model = train_model(
    ProtoFOMAML,
    train_loader=train_protomaml_loader,
    val_loader=val_protomaml_loader,
    outerLR=5e-4, innerLR=1e-3, outputLR=1e-2, steps=5, batchSize=8, warmupSteps=0
)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Global seed set to 42
/home/aksingh/miniconda3/envs/meta-learned-lines/lib/python3.10/site-packages/pytorch_lightning/trainer/configuration_validator.py:72: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(

  | Name        | Type               | Params
---------------------------------------------------
0 | metaLearner | PrototypeMetaModel | 108 M 
---------------------------------------------------
393 K     Trainable params
108 M     Non-trainable params
108 M     Total params
434.816   Total estimated model params size (MB)


Training ProtoFOMAML with parameters "batchSize":   8
"innerLR":     0.001
"outerLR":     0.0005
"outputLR":    0.01
"steps":       5
"warmupSteps": 0
Number of labels in the episode are 3 and lines are 1
inner loop training loss is 11.179638385772705 and accuracy is 0.8333333333333334
inner loop training loss is 11.231433868408203 and accuracy is 0.9166666666666666
inner loop training loss is 11.463162660598755 and accuracy is 0.6666666666666666
inner loop training loss is 11.127881526947021 and accuracy is 1.0
inner loop training loss is 12.115628480911255 and accuracy is 0.75
outer loop training accuracy is 0.25 and loss is 13.138765811920166
Number of labels in the episode are 2 and lines are 1
inner loop training loss is 4.020815849304199 and accuracy is 0.875
inner loop training loss is 3.639312505722046 and accuracy is 1.0
inner loop training loss is 3.6645801067352295 and accuracy is 1.0
inner loop training loss is 4.10959529876709 and accuracy is 0.875
inner loop training loss

/home/aksingh/miniconda3/envs/meta-learned-lines/lib/python3.10/site-packages/pytorch_lightning/loops/optimization/automatic.py:129: UserWarning: `training_step` returned `None`. If this was on purpose, ignore this warning...
  self.warning_cache.warn("`training_step` returned `None`. If this was on purpose, ignore this warning...")


Number of labels in the episode are 3 and lines are 1
inner loop training loss is 10.303834915161133 and accuracy is 0.75
inner loop training loss is 10.948730230331421 and accuracy is 0.8333333333333334
inner loop training loss is 10.399428606033325 and accuracy is 0.75
inner loop training loss is 10.181949615478516 and accuracy is 0.75
inner loop training loss is 10.074029684066772 and accuracy is 0.75
outer loop training accuracy is 0.3333333333333333 and loss is 13.953200817108154
Number of labels in the episode are 2 and lines are 1
inner loop training loss is 4.5719428062438965 and accuracy is 1.0
inner loop training loss is 4.424367427825928 and accuracy is 1.0
inner loop training loss is 4.390249252319336 and accuracy is 0.875
inner loop training loss is 4.502505779266357 and accuracy is 0.875
inner loop training loss is 4.437375068664551 and accuracy is 1.0
outer loop training accuracy is 0.625 and loss is 5.587550640106201
Number of labels in the episode are 2 and lines are 1

outer loop training accuracy is 0.75 and loss is 5.265951633453369
Number of labels in the episode are 3 and lines are 1
inner loop training loss is 11.425241470336914 and accuracy is 0.6666666666666666
inner loop training loss is 10.475454330444336 and accuracy is 0.75
inner loop training loss is 12.691738843917847 and accuracy is 0.5
inner loop training loss is 10.605561256408691 and accuracy is 0.75
inner loop training loss is 10.279190301895142 and accuracy is 0.75
outer loop training accuracy is 0.25 and loss is 14.24005937576294
Number of labels in the episode are 3 and lines are 1
inner loop training loss is 12.0438814163208 and accuracy is 0.5833333333333334
inner loop training loss is 11.628462791442871 and accuracy is 0.75
inner loop training loss is 11.320552825927734 and accuracy is 0.75
inner loop training loss is 10.822057485580444 and accuracy is 0.9166666666666666
inner loop training loss is 11.506050109863281 and accuracy is 0.5833333333333334
outer loop training accur

/home/aksingh/miniconda3/envs/meta-learned-lines/lib/python3.10/site-packages/pytorch_lightning/trainer/call.py:54: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


IsADirectoryError: [Errno 21] Is a directory: '/home/aksingh/Documents/meta-learned-lines'

In [ ]:
from training_datasets.GLUEMetaDataset import GLUEMetaDataset
from samplers.FewShotEpisodeSampler import FewShotEpisodeSampler
from samplers.FewShotEpisodeBatchSampler import FewShotEpisodeBatchSampler
import torch.utils.data as data

meta_ds = GLUEMetaDataset(k=4,numTasks=1000)

In [ ]:
from training.models.ProtoNet import ProtoNet
import torch.utils.data as data
from training_datasets.GLUEMetaDataset import GLUEMetaDataset
from training.trainer import train_model

from samplers.FewShotValidationEpisodeBatchSampler import FewShotValidationEpisodeBatchSampler
from samplers.FewShotValidationEpisodeSampler import FewShotValidationEpisodeSampler
from validation_datasets.ValidationDataset import ValidationDataset
from validation_datasets.GLUEMetaValidationDataset import GLUEMetaValidationDataset
import torch.utils.data as data

ds = ValidationDataset()

val_protomaml_sampler = FewShotValidationEpisodeBatchSampler(ds, kShot=4)
val_protomaml_loader = data.DataLoader(
    ds, batch_sampler=val_protomaml_sampler, collate_fn=val_protomaml_sampler.getCollateFunction(), num_workers=1
)

validation_dataset = GLUEMetaValidationDataset(k=8, numTasks=5000)
val_protomaml_loader = data.DataLoader(validation_dataset, num_workers=2)

train_protonet_loader = data.DataLoader(
    meta_ds, num_workers=4)

protomaml_model = train_model(
    ProtoNet,
    train_loader=train_protonet_loader,
    val_loader=val_protomaml_loader,
    metaLearningRate=5e-5, prototypeLearningRate=1e-2, steps=5, batchSize=8, warmupSteps=0
)